In [5]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "MLP-KTLim/llama-3-Korean-Bllossom-8B"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)
model.eval()

PROMPT = "You are a helpful AI assistant. Please answer the user's questions kindly. 당신은 유능한 AI 어시스턴트 입니다. 사용자의 질문에 대해 친절하게 답변해주세요."
instruction = "서울의 유명한 관광 코스를 만들어줄래?"

messages = [
    {"role": "system", "content": PROMPT},
    {"role": "user", "content": instruction},
]

enc = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt",
)
enc = enc.to(model.device)

eot = tokenizer.convert_tokens_to_ids("<|eot_id|>")
terminators = [tokenizer.eos_token_id] + ([eot] if eot is not None else [])

outputs = model.generate(
    input_ids=enc["input_ids"],
    attention_mask=enc.get("attention_mask", None),
    max_new_tokens=512,          # 처음엔 2048 말고 256~512 추천
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)

prompt_len = enc["input_ids"].shape[-1]
print(tokenizer.decode(outputs[0][prompt_len:], skip_special_tokens=True))


`torch_dtype` is deprecated! Use `dtype` instead!


Loading weights:   0%|          | 0/291 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [6]:
def textAugmentation(text):
    PROMPT = '''You are a helpful AI assistant. Please answer the user's questions kindly.
                당신은 문장을 재구성하는 전문가입니다. 주어진 문장과 의미가 동일한 새로운 문장을 생성해주세요.'''
    instruction = f"아래 문장과 동일한 의미를 갖는 새로운 문장을 하나만 만들어줘:\n\n'{text}'"
    
    messages = [
        {"role": "system", "content": PROMPT},
        {"role": "user", "content": instruction},
    ]
    
    enc = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt",
    )
    enc = enc.to(model.device)
    
    eot = tokenizer.convert_tokens_to_ids("<|eot_id|>")
    terminators = [tokenizer.eos_token_id] + ([eot] if eot is not None else [])
    
    outputs = model.generate(
        input_ids=enc["input_ids"],
        attention_mask=enc.get("attention_mask", None),
        max_new_tokens=512,          # 처음엔 2048 말고 256~512 추천
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )
    
    prompt_len = enc["input_ids"].shape[-1]

    
    return tokenizer.decode(outputs[0][prompt_len:], skip_special_tokens=True)

In [9]:
text = "길동경찰서입니다. 9시 40분 마트에 폭발물을 설치할거다. 네? 똑바로 들어 한번만 더 얘기한다. 장난전화 걸지 마시죠. 9시 40분 마트에 폭발물이 터지면 다 죽는거야. 장난전화는 업무방해죄에 해당됩니다. 판단은 너에게 달려있다. 길동경찰서에도 폭발물 터지면 꽤나 재미있겠지. 선생님 진정하세요. 난 이야기했어. 경고했다는 말이야."

In [10]:
print(textAugmentation(text))

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


길동경찰서입니다. 9시 40분 마트에 폭발물을 설치할 거예요. 네? 똑바로 들어 한번만 더 얘기하겠습니다. 장난전화는 업무 방해죄에 해당됩니다. 9시 40분 마트에 폭발물이 터지면 모두 죽을 것입니다. 판단은 당신에게 달려 있습니다. 길동경찰서에도 폭발물 터지면 꽤 재미있겠지요. 선생님, 진정하세요. 이야기를 했어요. 경고를 했으니 주의하세요.


In [11]:
text = "어이 거기 예?? 너 말이야 너. 이리 오라고 무슨 일. 너 옷 좋아보인다? 얘 돈 좀 있나봐 아니에요.돈 없어요 뒤져서 나오면 넌 죽는다 오늘 피시방 콜? 콜. 마지막 기회다. 있는거 다 내놔 정말 없어요"

In [12]:
print(textAugmentation(text))

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


'어이 거기 예?? 너 말이야 너. 이리 오라고 무슨 일. 너 옷 좋아보인다? 얘 돈 좀 있나봐 아니에요. 돈 없어요 뒤져서 나오면 넌 죽는다 오늘 피시방 콜? 콜. 마지막 기회다. 있는거 다 내놔 정말 없어요.'


In [35]:
import torch
print("cuda available:", torch.cuda.is_available())
print("torch cuda version:", torch.version.cuda)

cuda available: True
torch cuda version: 12.8


In [36]:
import torch
torch.cuda.empty_cache()
torch.cuda.ipc_collect()

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = 'Bllossom/llama-3.2-Korean-Bllossom-3B'

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)
instruction = "철수가 20개의 연필을 가지고 있었는데 영희가 절반을 가져가고 민수가 남은 5개를 가져갔으면 철수에게 남은 연필의 갯수는 몇개인가요?"

messages = [
    {"role": "user", "content": f"{instruction}"}
    ]

enc = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

print("model device:", next(model.parameters()).device)

terminators = [
    tokenizer.convert_tokens_to_ids("<|end_of_text|>"),
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

# outputs = model.generate(
#     input_ids,
#     max_new_tokens=1024,
#     eos_token_id=terminators,
#     do_sample=True,
#     temperature=0.6,
#     top_p=0.9
# )

outputs = model.generate(
    input_ids=enc["input_ids"],
    attention_mask=enc.get("attention_mask", None),
    max_new_tokens=512,          # 처음엔 2048 말고 256~512 추천
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)


prompt_len = enc["input_ids"].shape[-1]
print(tokenizer.decode(outputs[0][prompt_len:], skip_special_tokens=True))
# print(tokenizer.decode(outputs[0]))
# print(tokenizer.decode(outputs[0][input_ids.shape[-1]:]))
# print(tokenizer.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True))

`torch_dtype` is deprecated! Use `dtype` instead!


Loading weights:   0%|          | 0/254 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


model device: cuda:0
철수가 20개의 연필을 가지고 있었고, 영희가 절반을 가져갔으니, 영희가 가져간 연필은 20 / 2 = 10개입니다.

철수가 남은 연필은 20 - 10 = 10개입니다.

민수가 남은 5개를 가져갔으니, 철수가 남은 연필은 10 - 5 = 5개입니다.

따라서 철수가 남은 연필의 갯수는 5개입니다.


In [2]:
def textAugmentation(model, PROMPT, instruction):

    messages = [
        {"role": "system", "content": PROMPT},
        {"role": "user", "content": instruction},
    ]
    
    
    enc = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)
    
    terminators = [
        tokenizer.convert_tokens_to_ids("<|end_of_text|>"),
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]
    
    outputs = model.generate(
        input_ids=enc["input_ids"],
        attention_mask=enc.get("attention_mask", None),
        max_new_tokens=512,          # 처음엔 2048 말고 256~512 추천
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )
    
    prompt_len = enc["input_ids"].shape[-1]
    return tokenizer.decode(outputs[0][prompt_len:], skip_special_tokens=True)

In [71]:
text = "저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 발라보실래요? 아 진짜요? 안 그래도 선크림 필요해서 알아보던 중인데 한 번 발라 볼게요! 여기 한 번 발라보세요. 진짜 성분도 좋고 다 좋아요. 음. 성분이 좋다고 하셔서 좋은거 같기는 한데 제 피부에 맞지 않나봐요. 피부가 따끔거리네요. 이번에 진짜 열심히 연구해서 만든건데 피부가 많이 예민하신가봐요. 네 많이 예민해요. 그럼 많이 파시고 안녕히 계세요. 아니 저기요 돈 안내요? 네? 발라보는것도 돈 내야 하나요? 그럼 이거 누구한테 팔아요? 당신이 바른거를? 아니 먼저 발라 보시라고 하셨잖아요. 먼저 권유해놓고 사라고 강매하는거 갈취인거 몰라요? 내가 안 사도 된다고 말 한 적 있어? 그것도 모르고 바른걸 누구 탓 하나? 빨리 사 당신이 바른거 당신이 사야지 진짜 어이가 없어서 다른 사람들한텐 이렇게 갈취하지마세요. 화딱지나네"

PROMPT = """
    너는 한국어 대화 데이터 증강 전문가야.

    규칙
    1. 자연스러운 한국어로 생성
    """
instruction = f"의미를 유지해서 문장의 단어와 말투를 바꿔라.:\n\n'{text}'"


print(text)

저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 발라보실래요? 아 진짜요? 안 그래도 선크림 필요해서 알아보던 중인데 한 번 발라 볼게요! 여기 한 번 발라보세요. 진짜 성분도 좋고 다 좋아요. 음. 성분이 좋다고 하셔서 좋은거 같기는 한데 제 피부에 맞지 않나봐요. 피부가 따끔거리네요. 이번에 진짜 열심히 연구해서 만든건데 피부가 많이 예민하신가봐요. 네 많이 예민해요. 그럼 많이 파시고 안녕히 계세요. 아니 저기요 돈 안내요? 네? 발라보는것도 돈 내야 하나요? 그럼 이거 누구한테 팔아요? 당신이 바른거를? 아니 먼저 발라 보시라고 하셨잖아요. 먼저 권유해놓고 사라고 강매하는거 갈취인거 몰라요? 내가 안 사도 된다고 말 한 적 있어? 그것도 모르고 바른걸 누구 탓 하나? 빨리 사 당신이 바른거 당신이 사야지 진짜 어이가 없어서 다른 사람들한텐 이렇게 갈취하지마세요. 화딱지나네


In [72]:
import time
import torch

torch.cuda.synchronize()   # ✅ GPU 대기
start_time = time.time()

print(textAugmentation(model, PROMPT, instruction))

torch.cuda.synchronize()   # ✅ 다시 대기
end_time = time.time()

elapsed = end_time - start_time
print(f"⏱ GPU 포함 실행 시간: {elapsed:.3f}초")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 발라보실래요? 아 진짜요? 안 그래도 선크림 필요해서 알아보던 중인데 한 번 발라 볼게요. 여기 한 번 발라보세요. 진짜 성분도 좋고 다 좋아요. 음. 성분이 좋다고 하셔서 좋은거 같기는 한데 제 피부에 맞지 않나봐요. 피부가 따끔거리네요. 이번에 진짜 열심히 연구해서 만든건데 피부가 많이 예민하신가봐요. 네 많이 예민해요. 그럼 많이 파시고 안녕히 계세요. 아니 저기요 돈 안내요? 네? 발라보는것도 돈 내야 하나요? 그럼 이거 누구한테 팔아요? 당신이 바른거를? 아니 먼저 발라 보시라고 하셨잖아요. 먼저 권유해놓고 사라고 강매하는거 갈취인거 몰라요? 내가 안 사도 된다고 말 한 적 있어? 그도 모르고 바른걸 누구 탓 하나? 빨리 사 당신이 바른거 당신이 사야지 진짜 어이가 없어서 다른 사람들한텐 이렇게 갈취하지마세요. 화딱지나네.'
⏱ GPU 포함 실행 시간: 8.494초


In [69]:

text = "김 대리 승진해야지 부장님 그래도 이건 아닙니다. 사는게 다 그래. 나는 이렇게 안 한줄 알아? 그래도 전 제 소신을 지키고 싶습니다. 소신은 나가서 지켜요 여기 말고요 부장님. 난 말했고. 김 대리가 부처할거면 해. 근데 여긴 그런 곳 아냐 어쩌면 좋죠. 그걸 알면 내가 알려줘? 알아? 그냥 닥치는대로 해 알겠습니다."

PROMPT = """
    너는 한국어 대화 데이터 증강 전문가야.

    규칙
    1. 자연스러운 한국어로 생성
    """
instruction = f"의미를 유지해서 문장 길이를 줄여라.:\n\n'{text}'"


print(text)

김 대리 승진해야지 부장님 그래도 이건 아닙니다. 사는게 다 그래. 나는 이렇게 안 한줄 알아? 그래도 전 제 소신을 지키고 싶습니다. 소신은 나가서 지켜요 여기 말고요 부장님. 난 말했고. 김 대리가 부처할거면 해. 근데 여긴 그런 곳 아냐 어쩌면 좋죠. 그걸 알면 내가 알려줘? 알아? 그냥 닥치는대로 해 알겠습니다.


In [70]:
import time
import torch

torch.cuda.synchronize()   # ✅ GPU 대기
start_time = time.time()

print(textAugmentation(model, PROMPT, instruction))

torch.cuda.synchronize()   # ✅ 다시 대기
end_time = time.time()

elapsed = end_time - start_time
print(f"⏱ GPU 포함 실행 시간: {elapsed:.3f}초")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'김 대리가 승진하면 좋겠지만, 여기서 내 소신을 지키고 싶습니다. 소신은 여기서 말고요. 김 대리가 부처할 때 해. 여기서 닥치는대로 해. 알겠습니다.'
⏱ GPU 포함 실행 시간: 2.205초


In [68]:
text = "저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 발라보실래요? 아 진짜요? 안 그래도 선크림 필요해서 알아보던 중인데 한 번 발라 볼게요! 여기 한 번 발라보세요. 진짜 성분도 좋고 다 좋아요. 음. 성분이 좋다고 하셔서 좋은거 같기는 한데 제 피부에 맞지 않나봐요. 피부가 따끔거리네요. 이번에 진짜 열심히 연구해서 만든건데 피부가 많이 예민하신가봐요. 네 많이 예민해요. 그럼 많이 파시고 안녕히 계세요. 아니 저기요 돈 안내요? 네? 발라보는것도 돈 내야 하나요? 그럼 이거 누구한테 팔아요? 당신이 바른거를? 아니 먼저 발라 보시라고 하셨잖아요. 먼저 권유해놓고 사라고 강매하는거 갈취인거 몰라요? 내가 안 사도 된다고 말 한 적 있어? 그것도 모르고 바른걸 누구 탓 하나? 빨리 사 당신이 바른거 당신이 사야지 진짜 어이가 없어서 다른 사람들한텐 이렇게 갈취하지마세요. 화딱지나네"

PROMPT = """
    넌 영어 번역가야. 한국어를 영어로 번역해줘

    규칙
    1. 설명하지 말고 답변만 출력할 것
    2. 화자 간 관계는 바꾸지 말 것
    """
instruction = f"다음 대화 문장을 영어로 번역해줘. 출력은 따로 설명 없이 번역문만 출력해줘:\n\n'{text}'"


print(text)

저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 발라보실래요? 아 진짜요? 안 그래도 선크림 필요해서 알아보던 중인데 한 번 발라 볼게요! 여기 한 번 발라보세요. 진짜 성분도 좋고 다 좋아요. 음. 성분이 좋다고 하셔서 좋은거 같기는 한데 제 피부에 맞지 않나봐요. 피부가 따끔거리네요. 이번에 진짜 열심히 연구해서 만든건데 피부가 많이 예민하신가봐요. 네 많이 예민해요. 그럼 많이 파시고 안녕히 계세요. 아니 저기요 돈 안내요? 네? 발라보는것도 돈 내야 하나요? 그럼 이거 누구한테 팔아요? 당신이 바른거를? 아니 먼저 발라 보시라고 하셨잖아요. 먼저 권유해놓고 사라고 강매하는거 갈취인거 몰라요? 내가 안 사도 된다고 말 한 적 있어? 그것도 모르고 바른걸 누구 탓 하나? 빨리 사 당신이 바른거 당신이 사야지 진짜 어이가 없어서 다른 사람들한텐 이렇게 갈취하지마세요. 화딱지나네


In [44]:
import time
import torch

torch.cuda.synchronize()   # ✅ GPU 대기
start_time = time.time()

kor_to_eng_text = textAugmentation(model, PROMPT, instruction)

torch.cuda.synchronize()   # ✅ 다시 대기
end_time = time.time()

elapsed = end_time - start_time
print(f"⏱ GPU 포함 실행 시간: {elapsed:.3f}초")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


⏱ GPU 포함 실행 시간: 6.557초


In [46]:
PROMPT = """
    넌 한국어 번역가야. 영어를 한국어로 번역해줘
    """
instruction = f"다음 대화 문장을 한국어로 번역해줘:\n\n'{kor_to_eng_text}'"


print(kor_to_eng_text)

Is it too hot here? Do you want to try some sunscreen from our company? Really? I was just looking for some sunscreen anyway. Here, try it out. It's really good, and it's great for everyone. I mean, it's good because you said so, but it might not be good for your skin. It stings a bit. This one is really well-researched, but I guess you're sensitive. You are very sensitive. So, buy a lot of it, and goodbye. Are you okay? Do you need some money? Do you need to buy it? So, who are you going to sell this to? Are you going to sell the right one? You said to try it out, not to buy it. I don't understand why you're taking advantage of people like this. I never said I wouldn't buy it. You didn't know that, did you? It's not your fault. You should sell it yourself, not take advantage of others. It's really ridiculous.


In [48]:
import time
import torch

torch.cuda.synchronize()   # ✅ GPU 대기
start_time = time.time()

eng_to_kor_text = textAugmentation(model, PROMPT, instruction)

torch.cuda.synchronize()   # ✅ 다시 대기
end_time = time.time()

elapsed = end_time - start_time
print(f"⏱ GPU 포함 실행 시간: {elapsed:.3f}초")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


⏱ GPU 포함 실행 시간: 6.972초


In [49]:
print(eng_to_kor_text)

'이곳이 너무 따뜻하네요? 우리 회사의 солнscreen을 시도해 보세요?จริง으로? 나도 그냥 солнscreen을 찾고 있었습니다. 여기서 thử해 보세요. 정말 좋아요, 그리고 mọi人的 좋아요에 좋습니다. 말해도 좋지만, 당신의 피부에 좋지 않을 수 있습니다. 조금 쾌라지게 느껴집니다. 이 제품은 정말 잘 연구된 것 같지만, 당신이 너무 민감한 것 같아요. 당신이 매우 민감한 것 같아요. 그래서 많은 수량을 구입해 주세요, 그리고 goodbye. 당신이 okay하세요? 당신이 돈이 필요해? 당신이 구입해야 할 것인가? 이 제품을 whom에게 판매할 건가요? 이 제품을 올바른 제품으로 판매할 건가요? 당신이 시도해 보라고 말했어, 구입하지 말라고. 왜 사람들을 이 तरह 유혹하는 건지 이해 못합니다. 나는 구입하지 않겠다고 말한 건 아니잖아요. 당신은 그것을 알고 있지 않았어. 그것은 당신의 책임이 아니야. 당신이 자신이 판매해야 할 것 같아, 다른 사람들을 유혹하는 것이 아니야. 정말 무리스럽다.'


In [160]:
PROMPT = """
    너는 한국어 대화 데이터 증강 전문가야.

    규칙
    1. 대화는 총 4번 오고가게 작성.
    2. 자연스러운 한국어로 생성
    3. 설명 없이 요청한 답만 출력
    4. 화자 구분 없이 이어서 작성
"""
instruction = f"""
    두 사람이 대화하는 대화문을 하나 작성해줘.
"""


In [161]:
import time
import torch

torch.cuda.synchronize()   # ✅ GPU 대기
start_time = time.time()

print(textAugmentation(model, PROMPT, instruction))

torch.cuda.synchronize()   # ✅ 다시 대기

end_time = time.time()

elapsed = end_time - start_time
print(f"⏱ GPU 포함 실행 시간: {elapsed:.3f}초")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


이민호: 안녕하세요, 오늘은 무슨 계획이 있어요?
정미: 오늘은 친구들과 영화를 보러 가고요. 영화가 나중에 나와서 시간을 맞추게 해요.
이민호: 좋아요. 그럼 그 영화는 어떤 것인가요?
정미: 요즘에 많이 보이던 '아프리카'입니다. 그게 재미있을 거예요.
⏱ GPU 포함 실행 시간: 2.649초


In [3]:
PROMPT = """
    너는 두 사람이 일상적으로 대화하는 문장을 만들어주는 AI야.
"""

topic = "부모님과의 여행"

instruction = f"""
    대화하는 문장을 만들어줘.
    각 문장은 한 줄로 작성해.
    화자 이름은 쓰지 마.
    자연스럽고 현실적인 말투로 대화해.
    
    답변예시1
    이번 주말에 뭐 할까?
    영화 한 편 보는 건 어때?
    좋다, 팝콘도 사 가자.
    그럼 내가 예매할게.
    
    답변예시2
    오늘 회사에서 너무 피곤했어.
    무슨 일 있었어?
    회의가 계속 있어서 힘들었어.
    오늘은 일찍 쉬어야겠다.
    
    답변예시3
    요즘 어디 여행 가고 싶어?
    바다 있는 곳이 좋을 것 같아.
    부산이나 제주도 괜찮겠다.
    사진도 많이 찍자.

    
    아래 {topic} 주제로 비슷한 형식의 대화 한 개만 만들어.


"""

In [4]:
import time
import torch

torch.cuda.synchronize()   # ✅ GPU 대기
start_time = time.time()

print(textAugmentation(model, PROMPT, instruction))

torch.cuda.synchronize()   # ✅ 다시 대기

end_time = time.time()

elapsed = end_time - start_time
print(f"⏱ GPU 포함 실행 시간: {elapsed:.3f}초")

NameError: name 'model' is not defined